# 1. Data Retrieval

In [14]:
# Run this in a terminal
# !pip install -r requirements.txt

In [15]:
import requests
import pandas as pd
import time
import random
import re
import concurrent.futures
from bs4 import BeautifulSoup
import sqlite3

## Web Scraping

### Web Scraping Letterboxd to get the list of the top 500 movies in history

In [16]:
base_url = "https://letterboxd.com/reelstats/list/the-500-greatest-movies-of-all-time-according/detail/page/{}/"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
}

top_500_titles = []
for page_num in range(1, 6):
    url = base_url.format(page_num)
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")

    # Extract movie slugs
    movies = soup.select("ul.poster-list li div.poster")
    for movie in movies:
        title = movie.get("data-target-link", None)
        if title:
            parts = title.strip('/').split('/')
            if len(parts) >= 2:
                movie_slug = parts[1]
                movie_clean = movie_slug.replace('-', ' ').title()
                top_500_titles.append(movie_clean)
    print(f"✅ Scraped {len(movies)} movie titles from page {page_num}")

print(f"🎯 Total scraped: {len(top_500_titles)} movies")
print(top_500_titles[:10])

✅ Scraped 100 movie titles from page 1
✅ Scraped 100 movie titles from page 2
✅ Scraped 100 movie titles from page 3
✅ Scraped 100 movie titles from page 4
✅ Scraped 100 movie titles from page 5
🎯 Total scraped: 500 movies
['The Godfather', 'The Godfather Part Ii', 'Seven Samurai', '12 Angry Men', 'City Lights', 'The Good The Bad And The Ugly', 'The Shawshank Redemption', 'Psycho', 'Modern Times', 'Schindlers List']


In [17]:
print(top_500_titles[101:200])

['Three Colours Red', 'Yojimbo', 'Back To The Future', 'My Neighbor Totoro', 'In The Mood For Love', 'Princess Mononoke', 'Saving Private Ryan', 'Cinema Paradiso', 'La Jetee', 'The Wages Of Fear', 'Das Boot', 'Fight Club', 'Nights Of Cabiria', 'La Strada', 'Amadeus', 'Forrest Gump', 'Spider Man Into The Spider Verse', 'The Lion King', 'Inception', 'Whiplash 2014', 'The Shop Around The Corner', 'Rififi', 'Umberto D', 'Army Of Shadows', 'Blade Runner', 'Samurai Rebellion', 'Close Up', 'The Circus', 'Raiders Of The Lost Ark', 'Grand Illusion', 'A Clockwork Orange', 'Eternal Sunshine Of The Spotless Mind', 'A Woman Under The Influence', 'The Cranes Are Flying', 'Yi Yi', 'To Kill A Mockingbird', 'The Matrix', 'The Sting', 'The Mother And The Whore', 'Se7En', 'Early Summer', 'Werckmeister Harmonies', 'Coco 2017', 'Toy Story', 'It Happened One Night', 'Reservoir Dogs', 'Unforgiven', 'The Deer Hunter', 'The Young And The Damned', 'The Best Years Of Our Lives', 'The Leopard', 'Time Of The Gypsi

### Using OMDb's API to create the dataset

In [18]:
API_KEY = "93a674ac" 
OMDB_BASE_URL = "http://www.omdbapi.com/"

#to fetch data for one movie
def fetch_movie_data(title):
    params = {
        "t": title,
        "apikey": API_KEY
    }
    try:
        response = requests.get(OMDB_BASE_URL, params=params)
        if response.status_code == 200:
            data = response.json()
            if data.get("Response") == "True":
                return {
                    "Title": data.get("Title"),
                    "Year": data.get("Year"),
                    "Rated": data.get("Rated"),
                    "Runtime": data.get("Runtime"),
                    "Genre": data.get("Genre"),
                    "Director": data.get("Director"),
                    "Stars": data.get("Actors"),
                    "IMDb_Rating": data.get("imdbRating"),
                    "Metascore": data.get("Metascore"),
                    "Votes": data.get("imdbVotes"),
                    "Gross": data.get("BoxOffice"),
                    "Plot": data.get("Plot")
                }
            else:
                print(f"❗ Movie not found: {title}")
        else:
            print(f"❗ Request failed for: {title}")
    except Exception as e:
        print(f"❗ Error fetching {title}: {e}")
    return None

#for all movie metadata
movie_data = []

#manual corrections in movie titles
title_corrections = {
    "Schindlers List": "Schindler's List",
    "One Flew Over The Cuckoos Nest": "One Flew Over the Cuckoo's Nest",
    "Sunrise A Song Of Two Humans": "Sunrise: A Song of Two Humans",
    "Its A Wonderful Life": "It's a Wonderful Life",
    "8 Half": "8½",
    "Apur Sansar": "The World of Apu",
    "Witness For The Prosecution 1957": "Witness for the Prosecution",
    "Walle": "WALL·E",
    "Three Colours Red": "Three Colors: Red",
    "Coco 2017": "Coco",
    "Its Such A Beautiful Day": "It's Such a Beautiful Day",
    "The Cabinet Of Dr Caligari 1920": "The Cabinet of Dr. Caligari",
    "Underground 1995": "Underground",
    "Ivans Childhood": "Ivan's Childhood",
    "Faust 1926": "Faust",
    "Diabolique 1955": "Les Diaboliques",
    "Latalante": "L'Atalante",
    "Forbidden Games 1952": "Forbidden Games",
    "Howls Moving Castle": "Howl's Moving Castle",
    "Rosemarys Baby": "Rosemary's Baby",
    "Lavventura": "L'Avventura",
    "Three Colours Blue": "Three Colors: Blue",
    "Inside Out 2015": "Inside Out",
    "Where Is The Friends House": "Where Is My Friend's House?",
    "Nostalgia 1983": "Nostalghia",
    "Leclisse": "L'Eclisse",
    "In The Name Of The Father 1993": "In the Name of the Father",
    "The Maltese Falcon 1941": "The Maltese Falcon",
    "Pans Labyrinth": "Pan's Labyrinth",
    "Ben Hur 1959": "Ben-Hur",
    "Fireworks 1997": "Hana-bi",
    "My Night At Mauds": "My Night at Maud's",
    "Marriage Story 2019": "Marriage Story",
    "Intolerance Loves Struggle Throughout The Ages": "Intolerance: Love's Struggle Throughout the Ages",
    "My Left Foot The Story Of Christy Brown": "My Left Foot",
    "The Kings Speech": "The King's Speech",
    "Whos Afraid Of Virginia Woolf": "Who's Afraid of Virginia Woolf?",
    "The Gospel According To Matthew 1964": "The Gospel According to St. Matthew",
    "Kikis Delivery Service": "Kiki's Delivery Service",
    "All The Presidents Men": "All the President's Men",
    "Mommy 2014": "Mommy",
    "A Hard Days Night": "A Hard Day's Night",
    "Duck Soup 1933": "Duck Soup",
    "Carlitos Way": "Carlito's Way"
}
failed_movies = [] 
for idx, movie_title in enumerate(top_500_titles, 1):
    corrected_title = title_corrections.get(movie_title, movie_title)

    print(f"Fetching {idx}/{len(top_500_titles)}: {corrected_title}")
    data = fetch_movie_data(corrected_title)
    if data:
        movie_data.append(data)
    else:
        failed_movies.append(corrected_title)
    time.sleep(0.5)

df = pd.DataFrame(movie_data)
df.to_csv("imdb_top500_omdb.csv", index=False)
print("✅ Successfully saved imdb_top500_omdb.csv!")

Fetching 1/500: The Godfather
Fetching 2/500: The Godfather Part Ii
Fetching 3/500: Seven Samurai
Fetching 4/500: 12 Angry Men
Fetching 5/500: City Lights
Fetching 6/500: The Good The Bad And The Ugly
Fetching 7/500: The Shawshank Redemption
Fetching 8/500: Psycho
Fetching 9/500: Modern Times
Fetching 10/500: Schindler's List
Fetching 11/500: Pulp Fiction
Fetching 12/500: Rear Window
Fetching 13/500: One Flew Over the Cuckoo's Nest
Fetching 14/500: Apocalypse Now
Fetching 15/500: Tokyo Story
Fetching 16/500: Spirited Away
Fetching 17/500: Goodfellas
Fetching 18/500: Vertigo
Fetching 19/500: Singin In The Rain
Fetching 20/500: Sunset Boulevard
Fetching 21/500: Citizen Kane
Fetching 22/500: Harakiri
Fetching 23/500: Rashomon
Fetching 24/500: Once Upon A Time In The West
Fetching 25/500: Fanny And Alexander
Fetching 26/500: The Lord Of The Rings The Return Of The King
Fetching 27/500: Andrei Rublev
Fetching 28/500: The Passion Of Joan Of Arc


KeyboardInterrupt: 

- The dataset was compiled by scraping the top 500 movies from Letterboxd, which were distributed across five pages. 

- The scraping process was performed using Python's requests and BeautifulSoup libraries. 

- The scraped movie titles were then enriched using the OMDb API to retrieve structured metadata, including runtime, ratings, genres, directors, cast, and box office information. 

- All code for data retrieval is available in the `WebScraping+API-1.ipynb` notebook.

# Data Backups: Raw Data

In [19]:
#Loading data to check once
df = pd.read_csv('/Users/janhavisharma/Desktop/MSDS/2 SEM 2/Data Wrangling/Project/Final/Web-Scraping-EDA--IMDB-Project/imdb_top500_omdb.csv')

#### Method-1: CSV

Storing data in csv file <br>
Advantage- Good for Basic data sharing, helps in quick imports

In [20]:
df.to_csv('raw_data.csv', index=False)

#### Method-2: Excel

Storing data in excel file <br>
Advantage- Makes Data sharing with non-technical users easy

In [21]:
df.to_excel('movies_raw.xlsx', index=False)

#### Method-3: SQLite DB

Storing data in database storage using SQLite <br>
Advantage- Good for complex querying, ensures data integrity

In [22]:
conn = sqlite3.connect('movies_raw.db')
df.to_sql('movies', conn, index=False, if_exists='replace')
conn.close()

# We'll be performing EDA on this csv.